In [1]:
# Notebook to read in to the results of AiZynthFinders and parse reaction trees to json for further analysis

In [1]:
import pandas as pd
import json

import aizynthfinder.chem as chem
import aizynthfinder.analysis

from aizynthfinder.mcts.node import Node
from aizynthfinder.analysis import ReactionTree
from aizynthfinder.mcts.state import State
from aizynthfinder.chem import TreeMolecule
from aizynthfinder.context.collection import ContextCollection
from aizynthfinder.context.stock import StockException

In [12]:
import pandas as pd

data = pd.read_hdf('/data/localhost/not-backed-up/mokaya/exscientia/Retro/AiZynthFinder/aizynthfinder/data/Random/5/d12_test.hdf5', 'table')

In [13]:
print(data.head)

<bound method NDFrame.head of                                               target  search_time  \
0  CC(C)C(NC(=O)C1CSSC(C)(C)C(NC(=O)C(N)CC(=O)O)C...   121.755289   

   number_of_nodes  max_transforms  max_children  number_of_leafs  \
0              877              13            25              274   

   number_of_solved_leafs  top_score  is_solved  number_of_steps  \
0                       0   0.791673      False               13   

   number_of_precursors  number_of_precursors_in_stock  \
0                     6                              5   

                                 precursors_in_stock  \
0  CC(C)(C)O, OCc1ccccc1, CC(C)(C)O, OCc1ccccc1, ...   

                             precursors_not_in_stock policy_used_counts  \
0  COC(=O)C(NC(=O)C1CSSC(C)(C)C(NC(=O)C(CC(=O)O)N...     {'uspto': 876}   

                                          top_scores  \
0  0.7917, 0.7917, 0.7917, 0.7600, 0.7600, 0.7600...   

                                               trees  
0  [{'

In [14]:
solved_data = data.loc[(data.is_solved==True)]
unsolved_data = data.loc[(data.is_solved==False)]
all_solved = data.is_solved.values

In [15]:
true = []
false = []

for i in range(len(all_solved)):
    if all_solved[i] == True:
        true.append(i)
    else:
        false.append(i)


In [16]:
print('True:', (len(true)/len(all_solved))*100)
print('False: ', (len(false)/len(all_solved))*100)

True: 0.0
False:  100.0


In [17]:
from aizynthfinder.analysis import ReactionTree

all_trees = data.trees.values
all_solved_trees = solved_data.trees.values
all_unsolved_trees = unsolved_data.trees.values
first_molecule_tree = all_trees[0]

print(first_molecule_tree)



[{'type': 'mol', 'hide': False, 'smiles': 'CC(C)C(NC(=O)C1CSSC(C)(C)C(NC(=O)C(N)CC(=O)O)C(=O)NC(Cc2ccccc2)C(=O)NC(Cc2c[nH]c3ccccc23)C(=O)NC(CCCN)C(=O)NC(Cc2nc3ccccc3s2)C(=O)N1)C(=O)O', 'is_chemical': True, 'in_stock': False, 'children': [{'type': 'reaction', 'hide': False, 'smiles': '[N:1][CH3:2]>>O=C1c2ccccc2C(=O)[N:1]1[CH3:2]', 'is_reaction': True, 'metadata': {'template_hash': '7399a60fd7378a24dfa10a10ff432b29a0c80f1c6d7ac70d27cfe818', 'classification': 'NH deprotections', 'library_occurence': 1321, 'policy_probability': 0.003800000064074993, 'policy_name': 'uspto', 'template_code': 21226}, 'children': [{'type': 'mol', 'hide': False, 'smiles': 'CC(C)C(NC(=O)C1CSSC(C)(C)C(NC(=O)C(N)CC(=O)O)C(=O)NC(Cc2ccccc2)C(=O)NC(Cc2c[nH]c3ccccc23)C(=O)NC(CCCN2C(=O)c3ccccc3C2=O)C(=O)NC(Cc2nc3ccccc3s2)C(=O)N1)C(=O)O', 'is_chemical': True, 'in_stock': False, 'children': [{'type': 'reaction', 'hide': False, 'smiles': '[O:1][C:2]([CH3:3])=[O:4]>>CC(C)(C)[O:1][C:2]([CH3:3])=[O:4]', 'is_reaction': True, 

In [18]:
json_results = []
solved_json_results = []
unsolved_json_results = []

for i in all_trees:
    for itree, tree in enumerate(i):
        reaction_json = ReactionTree.from_dict(tree).to_json()
        json_results.append(reaction_json)

for i in all_unsolved_trees:
    for itree, tree in enumerate(i):
        unsolved_reaction_json = ReactionTree.from_dict(tree).to_json()
        unsolved_json_results.append(unsolved_reaction_json)

for i in all_solved_trees:
    for itree, tree in enumerate(i):
        solved_reaction_json = ReactionTree.from_dict(tree).to_json()
        solved_json_results.append(solved_reaction_json)



In [20]:
# save reaction trees to file

data = {
    'reaction trees': json_results
}

with open('/data/localhost/not-backed-up/mokaya/exscientia/Retro/AiZynthFinder/aizynthfinder/data/Random/5/d12_aug1_test.json', 'w') as outfile:
    json.dump(data, outfile)

In [10]:
print(json_results[0])

{
  "type": "mol",
  "hide": false,
  "smiles": "CC(C)C(NC(=O)C1CSSC(C)(C)C(NC(=O)C(N)CC(=O)O)C(=O)NC(Cc2ccccc2)C(=O)NC(Cc2c[nH]c3ccccc23)C(=O)NC(CCCN)C(=O)NC(Cc2nc3ccccc3s2)C(=O)N1)C(=O)O",
  "is_chemical": true,
  "in_stock": false,
  "children": [
    {
      "type": "reaction",
      "hide": false,
      "smiles": "([N:1][CH3:2].[N:3][CH3:4])>>(O=C(OCc1ccccc1)[N:1][CH3:2].O=C(OCc1ccccc1)[N:3][CH3:4])",
      "is_reaction": true,
      "metadata": {
        "template_hash": "a87cf7818afb30da6a71c2bcc16375a28c1f0e69a6202d24dbf0e17d",
        "classification": "NH deprotections",
        "library_occurence": 14,
        "policy_probability": 0.00279999990016222,
        "policy_name": "uspto",
        "template_code": 30820
      },
      "children": [
        {
          "type": "mol",
          "hide": false,
          "smiles": "CC(C)C(NC(=O)C1CSSC(C)(C)C(NC(=O)C(CC(=O)O)NC(=O)OCc2ccccc2)C(=O)NC(Cc2ccccc2)C(=O)NC(Cc2c[nH]c3ccccc23)C(=O)NC(CCCNC(=O)OCc2ccccc2)C(=O)NC(Cc2nc3ccccc3s2)